Retrieve experiment results

In [1]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [32]
basedirs = [join(get_output_dir(), 'predict_multi', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        seed = config['seed']
        model = config['model']
        source = config['source']
        score = info['score']
        rank = info['rank']
        n_components = config['n_components']
        latent_dropout_rate = config['latent_dropout_rate']
        normalized = config['with_std']
        res = {'datasets': datasets, 'n_components': n_components, 'latent_dropout_rate': latent_dropout_rate, 'seed': seed, 'rank': rank,
               'source': source, 'normalized': normalized,
              'model': model}
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

FileNotFoundError: [Errno 2] No such file or directory: '/home/arthur/output/cogspaces/predict_multi/32/run'

Compare results with trace norm, with and without transfer

In [2]:
df_agg = res.groupby(by=['normalized', 'source', 'model', 'datasets', 'alpha', 'beta']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)


dfs = {}
for dataset in ['brainomics', 'archi']:
    this_df = df_agg.query("datasets in ['%s__hcp', '%s']" % (dataset, dataset))
    test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in [dataset, 'hcp']) / 2
    this_df = this_df.assign(test_mean=test_mean)
    indices = this_df[('test_%s' % dataset, 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
    this_df = this_df.loc[indices.values]
    dfs[dataset] = this_df

NameError: name 'res' is not defined

Compare results with 3 datasets

In [3]:
this_df

NameError: name 'this_df' is not defined

In [4]:
indices.values

NameError: name 'indices' is not defined

In [5]:
this_df = df_agg.query("datasets == 'brainomics__archi__hcp'")
test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in ['brainomics', 'archi', 'hcp']) / 3
this_df = this_df.assign(test_mean=test_mean)
indices = this_df[('test_mean', 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
this_df = this_df.loc[indices.values]
dfs['archi+brainomics+hcp'] = this_df

NameError: name 'df_agg' is not defined

In [6]:
result_df = pd.concat(dfs.values(), keys=dfs.keys(), names=['target'])

NameError: name 'dfs' is not defined

In [7]:
result_df

NameError: name 'result_df' is not defined

In [8]:
df_agg.query("datasets == 'brainomics__archi__hcp'")

NameError: name 'df_agg' is not defined

In [9]:
df

NameError: name 'df' is not defined